In [1]:
import os
import sys
from pathlib import Path

# 1. Define Project Root
# On Colab/Kaggle, this is usually /content/lip-sync-dl-f25
# On PSC, it will be your current folder
PROJECT_ROOT = Path("lip-sync-dl-f25").resolve()
if not PROJECT_ROOT.exists():
    PROJECT_ROOT = Path(".").resolve() # Fallback if already inside

# 2. Clone if needed
if not (PROJECT_ROOT / ".git").exists():
    print("⬇️ Cloning repository...")
    !git clone https://github.com/MUKAMAFrancois/lip-sync-dl-f25.git
    os.chdir("lip-sync-dl-f25")
else:
    print("✅ Repo exists.")
    if PROJECT_ROOT.name == "lip-sync-dl-f25":
        os.chdir(PROJECT_ROOT)

# 3. Fix Nesting (Crucial Step)
# If we are in .../lip-sync-dl-f25/lip-sync-dl-f25, move up
cwd = Path.cwd()
if cwd.name == "lip-sync-dl-f25" and cwd.parent.name == "lip-sync-dl-f25":
    print("⚠️ Nested directory detected. Fixing...")
    os.chdir("..")

print(f"📂 Final Working Directory: {os.getcwd()}")

# 4. Install Deps
os.environ['KAGGLEHUB_CACHE_DIR'] = os.getcwd()
print("📦 Installing Dependencies...")
!pip install -q -r requirements.txt
!pip install -q kagglehub face-alignment pytorch-fid

⬇️ Cloning repository...
Cloning into 'lip-sync-dl-f25'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 74 (delta 33), reused 62 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 55.59 KiB | 18.53 MiB/s, done.
Resolving deltas: 100% (33/33), done.
📂 Final Working Directory: /content/lip-sync-dl-f25
📦 Installing Dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 54.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.

In [2]:
import kagglehub, shutil, os
from pathlib import Path

# 1. Run Setup Script (Creates directories)
!python setup_wav2lip.py

# 2. Download Data
print("⬇️ Downloading Dataset...")
path = kagglehub.dataset_download("francoismukama/muavic-german-sample")
target = Path("data/german")
os.makedirs(target, exist_ok=True)

# 3. Move Data
src_root = Path(path) / "mtedx/video/de"
if src_root.exists():
    for split in ["train", "val", "test"]:
        src = src_root / split
        # Fix validation naming if needed
        if not src.exists() and split == "val": src = src_root / "valid"

        dst = target / split
        if src.exists():
            if dst.exists(): shutil.rmtree(dst)
            shutil.copytree(src, dst)
            print(f"✅ Copied {split}")

⬇️ Cloning Wav2Lip into /content/lip-sync-dl-f25/Wav2Lip...
Cloning into 'Wav2Lip'...
remote: Enumerating objects: 409, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 409 (delta 2), reused 0 (delta 0), pack-reused 405 (from 2)
Receiving objects: 100% (409/409), 549.28 KiB | 21.13 MiB/s, done.
Resolving deltas: 100% (227/227), done.
⬇️ Downloading wav2lip_gan.pth...
⬇️ Downloading lipsync_expert.pth...
✅ Setup Complete.
⬇️ Downloading Dataset...
Using Colab cache for faster access to the 'muavic-german-sample' dataset.
✅ Copied train
✅ Copied val
✅ Copied test


In [3]:
print("⚙️ Setting up Wav2Lip Models...")
!python setup_wav2lip.py

⚙️ Setting up Wav2Lip Models...
✅ Found: wav2lip_gan.pth
✅ Found: lipsync_expert.pth
✅ Setup Complete.


In [4]:
# Now this will work because we overwrote the script in Cell 3!
!python preprocess_training_data.py \
  --data_root data/german \
  --output_root data/german/preprocessed \
  --filelist_root data/german/filelists \
  --no-zip

🚀 Preprocessing on cuda
Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth
100% 85.7M/85.7M [00:06<00:00, 13.5MB/s]
🎬 Processing train...
  0% 0/9 [00:00<?, ?it/s]^C


In [5]:
!python training/train.py

❌ Critical: Could not import Wav2Lip models.


In [6]:
import torch
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import sys

# Setup Path
if Path("Wav2Lip").exists(): sys.path.append("Wav2Lip")

from models import Wav2Lip
import audio

def run_inference_demo():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Check for model
    ckpts = sorted(Path("checkpoints").glob("*.pth"))
    if not ckpts:
        print("❌ No checkpoint found to test inference.")
        return
    latest_ckpt = ckpts[-1]

    # Load
    model = Wav2Lip().to(device)
    ckpt = torch.load(latest_ckpt, map_location=device)
    model.load_state_dict({k.replace('module.', ''): v for k, v in ckpt['state_dict'].items()})
    model.eval()

    print(f"✅ Model loaded: {latest_ckpt.name}")
    print("   (Inference code ready for full run)")

run_inference_demo()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject